In [ ]:
"""
!pip install python-dotenv
!pip install google-generativeai
"""

In [2]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

In [5]:
load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

In [3]:
def read_file_lines(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.readlines()
    except FileNotFoundError:
        print(f"[ERROR] File not found: {file_path}")
        return []
    except Exception as e:
        print(f"[ERROR] An error occurred: {e}")
        return []

def find_text_files(base_directory):
    text_files = {}
    target_directory = os.path.join(base_directory, 'src')
    for root, _, files in os.walk(target_directory):
        for file in files:
            if file.endswith('.txt'):
                folder = os.path.relpath(root, target_directory).replace("\\", "/")
                file_path = os.path.join(root, file).replace("\\", "/")
                text_files.setdefault(folder, []).append(file_path)
    return text_files

def log_grouped_files(grouped_files):
    for folder, files in grouped_files.items():
        print(f"\nFolder : {folder.upper()}")
        for idx, file_path in enumerate(files, start=1):
            print(f"{idx}. {file_path}")
    print("\n\n\n")

def generate_output(file_path, output_directory, folder_name, logged_folders):
    if folder_name not in logged_folders:
        print(f"\n################## {folder_name.upper()} ##################")
        logged_folders.add(folder_name)

    file_name = os.path.splitext(os.path.basename(file_path))[0]
    output_path = os.path.join(output_directory, f"{file_name}.md")

    if os.path.exists(output_path):
        print(f"[SKIPPED] File already exists: {file_name}")
        return

    os.makedirs(output_directory, exist_ok=True)

    print(f"\n~~~~~ {folder_name.upper()} - {file_name}.txt ~~~~~\n")
    print(f"Processing file: {file_path}")
    print(f"Output will be saved to: {output_path}\n")

    heading = f"# {folder_name}/{file_name}\n\n"
    questions = read_file_lines(file_path)
    if not questions:
        return

    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(heading)

        for idx, question in enumerate(questions, start=1):
            question_text = question.strip()
            if question_text:
                print(f" {question_text} ")
                try:
                    response = model.generate_content(
                        f"Give a detailed explanation of {question_text} in the form of paragraphs and points. "
                        f"It should be substantial, about 500 words."
                    )
                    file.write(f"## {question_text}\n\n")
                    file.write(response.text + "\n\n")
                    print(f"[SUCCESS]")
                except Exception as e:
                    print(f"[ERROR] - {e}")

def main():
    base_directory = 'aat_data'
    grouped_files = find_text_files(base_directory)

    log_grouped_files(grouped_files)

    logged_folders = set()  

    for folder_name, files in grouped_files.items():
        for file_path in files:
            relative_path = os.path.relpath(file_path, os.path.join(base_directory, 'src'))
            folder_name = os.path.dirname(relative_path).replace("\\", "/")
            output_directory = os.path.join(base_directory, 'out', folder_name)
            generate_output(file_path, output_directory, folder_name, logged_folders)

In [4]:
main()
print("\n\n ### [ COMPLETED ] ###")


Folder : CHAN
1. aat_data/src/chan/6602cs.txt
2. aat_data/src/chan/6602dnn.txt
3. aat_data/src/chan/6602es.txt
4. aat_data/src/chan/6602pda.txt
5. aat_data/src/chan/6602vr.txt

Folder : PERSON_A_[EXAMPLE]
1. aat_data/src/person_a_[example]/sample_1_[example].txt

Folder : PERSON_B_[EXAMPLE]
1. aat_data/src/person_b_[example]/sample_2_[example].txt





################## CHAN ##################

~~~~~ CHAN - 6602cs.txt ~~~~~

Processing file: aat_data/src/chan/6602cs.txt
Output will be saved to: aat_data\out\chan\6602cs.md

 1.Explain how cognitive science principles have contributed to the development of AI technologies, such as natural language processing and machine learning 
[SUCCESS]
 2.How do experts approach problem solving differently from novices? Provide an example to illustrate your point. 
[SUCCESS]
 3.How can motivation be cultivated and sustained over the long term, and how does it influence the effectiveness of various learning strategies, from goal setting to time mana